# GRO830 - Démonstration de l'algorithme A*

Vous avez déjà lu la description de l'algorithme A* dans le livre *Introduction to Autonomous Mobile Robot*. Ce document a pour objectif de faire une démonstration de son fonctionnement autant sur un graphe qu'une carte à cellules fixes.

## Version Graphe - Préambule

Tout d'abord, nous allons mettre en place quelques éléments pour définir notre graphe. Le tableau **nodes** contiendra les identifiants des noeuds (que des entiers pour l'instant) et le tableau **edges** des tuples définissant les liens entre ces noeuds. Les liens sont tous bidirectionnels.

In [ ]:
g_nodes = ['MTL', 'BMT', 'SHE', 'DMV', 'QUE'] # Abbréviations pour Montréal, Bromont, Sherbrooke, Drummondville et Québec

g_edges = [('MTL', 'BMT'),
           ('BMT', 'SHE'),
           ('SHE', 'DMV'),
           ('MTL', 'DMV'),
           ('DMV', 'QUE')]

# Les distances et positions sont approximatives :
g_dists = [75, 75, 75, 100, 150]

g_pos = {
    'MTL': (0,0),
    'BMT': (50, 0),
    'SHE': (100, 0),
    'DMV': (100, 50),
    'QUE': (200, 100)
}

Nous pouvons également dessiner ce graphe à l'aide de **networkx** et **matplotlib**. Les positions indiquées ne sont pas respectées à la lettre, mais nous retrouvons tout de même la structure logique du graphe :

In [ ]:
def draw_graph(nodes, edges, dists, pos):
    import networkx as nx
    import matplotlib.pyplot as plt
    graph = nx.Graph()
    graph.add_nodes_from(nodes)
    graph.add_edges_from(edges)
    
    nx.draw_networkx(graph, pos, node_size=1000, node_color="#EEEEEE")
    nx.draw_networkx_edge_labels(graph, pos, dict(zip(edges, dists)))

    # Nécessaire pour agrandir la zone de dessin :
    l,r = plt.xlim()
    t,b = plt.ylim()
    plt.xlim(l-20, r+20)
    plt.ylim(t-10, b+10)
    
    plt.axis('off')
    plt.show()
    
draw_graph(g_nodes, g_edges, g_dists, g_pos)

On peut maintenant mettre en oeuvre l'algorithme. Pour fonctionner, l'algorithme A* a besoin de quelques fonctions : le coût de déplacement entre deux noeuds, les voisins de chaque noeuds et une heuristique permettant d'estimer la distance jusqu'à la cible.

Commençons par le coût entre deux noeuds. Nous avons déjà cette information dans le tableau de distance, mais nous allons l'exprimer par une fonction :

In [ ]:
g_costs = dict(zip(g_edges, g_dists)) # Création d'un dictionnaire associant les liens à des distances.
def g_cost(node_a, node_b):
    edge   = (node_a, node_b)
    # Il faut aussi vérifier le lien inverse vu que nous sommes bidirectionnels :
    edge_r = (node_b, node_a)
    
    if (edge in g_costs):
        return g_costs[edge]
    elif (edge_r in g_costs):
        return g_costs[edge_r]
    else:
        # ERROR!
        return -1
    
    #assert edge in g_costs, "g_cost: no edge between %s and %s"%(str(node_a), str(node_b))
    
    
    return g_costs[edge]
        
print("Le coût entre MTL et BMT est %d."%(g_cost('MTL', 'BMT')))

Nous avons également besoin d'une fonction qui retourne la liste des voisins pour un noeud. Nous avons déjà la liste des liens, mais nous devons effectuer une recherche :

In [ ]:
def g_neighbors(node):
    assert node in g_nodes, "g_neighbors: node unknown"
    
    neighbors = []
    for (a,b) in g_edges:
        if (a == node):
            neighbors.append(b)
        elif (b == node):
            neighbors.append(a)
    return neighbors
            
print("Les voisins de SHE sont : %s."%(str(g_neighbors("SHE"))))

Finalement, nous avons besoin de la fonction heuristique $H$. Nous pourrions utiliser une matrice enregistrant ces distances à voi d'oiseau, mais nous allons utiliser les positions approximatives :

In [ ]:
def g_h(node_a, node_b):
    from math import sqrt
    assert node_a in g_pos, "g_h: %s not in g_pos"%(node_a)
    assert node_b in g_pos, "g_h: %s not in g_pos"%(node_b) 
    
    pos_a = g_pos[node_a]
    pos_b = g_pos[node_b]
    
    return sqrt((pos_a[0]-pos_b[0])**2 + (pos_a[1] - pos_b[1])**2)

print("H('MTL', 'QUE') = %d"%(g_h('MTL', 'QUE')))

Nous avons donc tous les éléments nécessaires pour mettre en place la fonction de l'algorithme. Celle-ci s'occupera de générer le contenu des fonctions $F$ et $G$ et retournera le chemin le plus court (s'il existe) :

In [ ]:
def astar(start, goal, c_fun, n_fun, h_fun):
    # Cherche le chemin le plus court entre start et goal à l'aide de l'algorithme A*.
    # Utilise les fonctions :
    #   c_fun(edge): retourne le coût de parcours du lien edge
    #   n_fun(node): retourne les voisins du noeud node
    #   h_fun(node_a, node_b): retourne la valeur de l'heuristique du coût de parcours entre node_a et node_b
    
    from math import inf # Valeur infinie
    
    search_set = [start] # Espace de recherche, ne contient que le noeud de départ pour l'instant.
    
    # Dictionnaire contenant le plus bas coût réel du chemin (G) jusqu'au noeud spécifié.
    g = {}
    # On initialise avec g = 0 pour le départ
    g[start] = 0
    
    # Dictionnaire contenant la plus basse valeur de la fonction f(node) pour chaque noeud.
    f = {} 
    # On initialise f(start) avec g (qui est 0) et l'heuristique jusqu'à la fin.
    f[start] = g[start] + h_fun(start, goal)
   
    # Dictionnaire qui conserve pour chaque noeud la provenance permettant la valeur f la plus faible.
    # Utilisé pour reconstruire le chemin lorsque l'objectif est atteint.
    from_node = {}
    # On initialise avec le noeud de départ sans provenance (None).
    from_node[start] = None
    
    while (len(search_set) > 0):
        # On trouve le noeud ayant la plus petite valeur f dans l'ensemble de recherche :
        min_f = inf
        min_n = None
        for n in search_set:
            assert n in f, "Error : %s not in F!"%(str(n))
            if f[n] < min_f:
                min_f = f[n]
                min_n = n
        
        # On retire le noeud ayant le plus petit 'f' et on poursuit avec lui :
        current = min_n
        search_set.remove(min_n)
        
        # Si le noeud en cours est l'objectif, c'est qu'aucun autre noeud dans l'espace de recherche n'a
        # un meilleur potentiel du chemin le plus court. On reconstruit le chemin ensuite. 
        if (current == goal):
            path_r = [goal]
            previous = from_node[goal]
            while (previous is not None):
                path_r.append(previous)
                previous = from_node[previous]             
            path_r.reverse() # On remet la liste dans le bon ordre
            return path_r
        
        ns = n_fun(current) # Les voisins du noeud en cours
        for n in ns:
            # Pour chaque voisin, on calcule sa fonction f et on l'ajoute à l'ensemble de recherche seulement si
            # la valeur de g est plus basse que celle déjà connue pour ce noeud
            g_n = g[current] + c_fun(current, n)
            f_n = g[current] + g_n + h_fun(n, goal)
            if ((g_n not in g) or (g_n < g[n])):
                from_node[n] = current
                g[n] = g_n
                f[n] = f_n
                if (n not in search_set):
                    search_set.append(n)
   
    # Nous avons vidé l'espace de recherche sans trouver de solution. Retourner une liste vide.
    return []

res = astar('MTL', 'SHE', g_cost, g_neighbors, g_h)